# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_train_data[-5:]

array([[5795, 2035,    0,    2],
       [5795, 2040,    0,    1],
       [5795, 2272,    0,    1],
       [5795, 2342,    0,    1],
       [5795, 2370,    0,    1]])

In [7]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 258360
Nr of rows in test file 117777


In [8]:
print('Nr of emails in training file', np.unique(sparse_train_data[:, 0]).size)

Nr of emails in training file 4013


In [9]:
print('Nr of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Nr of emails in test file 1724


### How to create an Empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [ ]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
sparse_train_data[10:13]

array([[ 0, 71,  1,  4],
       [ 0, 88,  1,  1],
       [ 0, 94,  1,  1]])

In [ ]:
sparse_train_data[10][3]

4

# Create a Full Matrix from a Sparse Matrix

In [ ]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix
    

In [ ]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 12.9 s, sys: 344 ms, total: 13.3 s
Wall time: 15.7 s


In [ ]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,0,0,0,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,2,2,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5791,0,2,0,1,1,0,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,2,2,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5795,0,1,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [ ]:
# Challenge: Calculate the probability of spam - the percent of spam messages in the training
# dataset. Store this value in a variable called prob_spam

In [ ]:
full_train_data.CATEGORY.size

4014

In [ ]:
full_train_data.CATEGORY.sum()

1249

In [ ]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

Probability of spam is 0.3111609367214748


## Total Number of Words / Tokens

In [ ]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4014,)

In [ ]:
email_lengths[:5]

DOC_ID
0     50
1     76
2     87
3     76
4    137
dtype: int64

In [ ]:
total_wc = email_lengths.sum()
total_wc

445838

## Number of Tokens in Spam & Ham Emails

**Challenge** Create a subset of the email_lengths series that only contains the spam messages. Call the subset spam_lengths. Then count the total number of words that occur in spam emails. 

Do the same for the non-spam emails. Create a subset called ham_lengths. Then count the total number of words that occur in the ham emails. 

In [ ]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1249,)

In [ ]:
spam_wc = spam_lengths.sum()
spam_wc

195664

In [ ]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [ ]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [ ]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

250174

In [ ]:
spam_wc + nonspam_wc - total_wc

0

In [ ]:
# Challenge: Can you figure out if spam emails or non-spam emails tend to be longer? Which
# category contains more tokens? Take a guess and then verify your intuition using the dataset

In [ ]:
print('Average nr of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.3f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 157
Average nr of words in ham emails 90.479


## Summing the Tokens Occuring in Spam

In [ ]:
full_train_features.shape

(4014, 2500)

In [ ]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,2,0,0,3,2,2,2,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1887,0,0,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,2,0,1,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,3,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1895,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_spam_tokens.shape

(1249, 2500)

In [ ]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [ ]:
summed_spam_tokens.tail()

2495     3
2496    19
2497    20
2498    33
2499     6
dtype: int64

In [ ]:
# Challenge: Repeat this process for the ham messages. Sum the tokens that occur in the 
# non-spam messages. Store the values in a variable called summed_ham_tokens. 

## Summing the Tokens Occuring in Ham

In [ ]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [ ]:
summed_ham_tokens.shape

(2500,)

In [ ]:
summed_ham_tokens.tail()

2495    35
2496    22
2497    19
2498     2
2499    25
dtype: int64

In [ ]:
train_ham_tokens[2499].sum() + 1

25

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [ ]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.009038
1    0.004885
2    0.006828
3    0.010582
4    0.006802
dtype: float64

In [ ]:
prob_tokens_spam.sum()

1.0000000000000075

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [ ]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0000000000000036

# P(Token) - Probability that Token Occurs 

In [ ]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [ ]:
prob_tokens_all.sum()

1.0000000000000024

# Save the Trained Model

In [ ]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

**Challenge:** Create a full matrix from the sparse_test_data. Time the function call. How long does it take? Separate the features and the target values. Save these as separate .txt files: a TEST_TARGET_FILE and a TEST_FEATURE_MATRIX file. 

In [ ]:
sparse_test_data.shape

(110307, 4)

In [ ]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

CPU times: user 5.11 s, sys: 143 ms, total: 5.25 s
Wall time: 5.39 s


In [ ]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [ ]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)